In [2]:
import tensorflow as tf
import numpy as np 
import csv
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [17]:
dataset_path = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
NUM_CLASSES = 4

In [18]:
X_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [20]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         (None, 42)                0         
                                                                 
 dense_3 (Dense)             (None, 20)                860       
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 4)                 44        
                                                                 
Total params: 1,114
Trainable params: 1,114
Non-trainable params: 0
_________________________________________________________________


In [21]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [22]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/10
45/57 [======================>.......] - ETA: 0s - loss: 1.3095 - accuracy: 0.3392 
Epoch 1: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
57/57 [==============================] - 3s 45ms/step - loss: 1.2941 - accuracy: 0.3518 - val_loss: 1.2021 - val_accuracy: 0.4353
Epoch 2/10
37/57 [==================>...........] - ETA: 0s - loss: 1.1942 - accuracy: 0.4139
Epoch 2: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
57/57 [==============================] - 0s 2ms/step - loss: 1.1829 - accuracy: 0.4187 - val_loss: 1.0944 - val_accuracy: 0.5530
Epoch 3/10
37/57 [==================>...........] - ETA: 0s - loss: 1.1289 - accuracy: 0.4683
Epoch 3: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
57/57 [==============================] - 0s 2ms/step - loss: 1.1154 - accuracy: 0.4710 - val_loss: 1.0033 - val_accuracy: 0.6525
Epoch 4/10
34/57 [================>.............] - ETA: 0s - loss: 1.0691 - accuracy: 0.5179
Epoch 4

In [24]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 [==============================] - 0s 984us/step - loss: 0.5694 - accuracy: 0.8521


In [25]:
model = tf.keras.models.load_model(model_save_path)

In [27]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))
print(y_test[0])

1/1 [==============================] - 0s 21ms/step
[0.7998339  0.12271202 0.02541777 0.05203631]
0
0
